In [ ]:
import cv2
import sys

import pickle
import pandas as pd

import plotly.express as px

import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import DataLoader

In [ ]:
sys.path.append('..')

In [ ]:
from dataset import *

#### Config

In [ ]:
dataset_path = '../../change_detection_dataset/SN7_buildings/train'
masks_path = '../../change_detection_dataset/SN7_masks'

train_df_path = '../../change_detection_dataset/dataset_1/train.csv'
valid_df_path = '../../change_detection_dataset/dataset_1/valid.csv'
test_df_path = '../../change_detection_dataset/dataset_1/test.csv'

### Function

In [ ]:
def get_changes(dataset):
    change_per_combination = []

    dataloader = DataLoader(dataset,
                            batch_size=4,
                            num_workers=4)
    
    for _, _, changes in tqdm(dataloader):
        for change in changes:
            change_per_combination.append(change.sum().numpy().item())

    return change_per_combination

In [ ]:
train_dataset = ChangeDetectionDataset(dataset_path=dataset_path,
                                       masks_path=masks_path,
                                       df_path=train_df_path,
                                       training_mode=False,
                                       return_masks=False)

valid_dataset = ChangeDetectionDataset(dataset_path=dataset_path,
                                       masks_path=masks_path,
                                       df_path=valid_df_path,
                                       training_mode=False,
                                       return_masks=False)

test_dataset = ChangeDetectionDataset(dataset_path=dataset_path,
                                      masks_path=masks_path,
                                      df_path=test_df_path,
                                      training_mode=False,
                                      return_masks=False)

### Training dataset

In [ ]:
change_per_combination_train = np.array(get_changes(train_dataset))
change_per_combination_valid = np.array(get_changes(valid_dataset))
change_per_combination_test = np.array(get_changes(test_dataset))

In [ ]:
train_change_df = pd.DataFrame(change_per_combination_train, columns=['train'])
valid_change_df = pd.DataFrame(change_per_combination_valid, columns=['valid'])
test_change_df = pd.DataFrame(change_per_combination_test, columns=['test'])

In [ ]:
pd.concat([train_change_df.describe(percentiles=[0.25, 0.50, 0.75, 0.95, 0.99]), 
           valid_change_df.describe(percentiles=[0.25, 0.50, 0.75, 0.95, 0.99]),
          test_change_df.describe(percentiles=[0.25, 0.50, 0.75, 0.95, 0.99])], axis=1)

In [ ]:
dataset = train_dataset
change_per_combination = change_per_combination_train

AOI_list = [x[0] for x in dataset.combinations_list]

AOI_first_appearance = [AOI_list.index(x) for x in set(AOI_list)]

fig, axs = plt.subplots(figsize=(300, 10))

plt.plot(range(len(change_per_combination)), change_per_combination)
plt.vlines(AOI_first_appearance, ymin=0, ymax=np.max(change_per_combination), linestyles='dashed', color='C1')

In [ ]:
dataset = valid_dataset
change_per_combination = change_per_combination_valid

AOI_list = [x[0] for x in dataset.combinations_list]

AOI_first_appearance = [AOI_list.index(x) for x in set(AOI_list)]

fig, axs = plt.subplots(figsize=(300, 10))

plt.plot(range(len(change_per_combination)), change_per_combination)
plt.vlines(AOI_first_appearance, ymin=0, ymax=np.max(change_per_combination), linestyles='dashed', color='C1')

In [ ]:
dataset = test_dataset
change_per_combination = change_per_combination_test

AOI_list = [x[0] for x in dataset.combinations_list]

AOI_first_appearance = [AOI_list.index(x) for x in set(AOI_list)]

fig, axs = plt.subplots(figsize=(300, 10))

plt.plot(range(len(change_per_combination)), change_per_combination)
plt.vlines(AOI_first_appearance, ymin=0, ymax=np.max(change_per_combination), linestyles='dashed', color='C1')

In [ ]:
len(train_dataset.combinations_list)

In [ ]:
with open('../train_change_per_combination.pickle', 'wb') as f:
    pickle.dump(change_per_combination_train, f)